In [9]:
import  os
import  json
import  pickle
import  matplotlib
import  tensorflow         as      tf
import  matplotlib.pyplot  as      plt
import  pylab              as      pl
import  numpy              as      np

In [10]:
from    tensorflow.keras.models               import  model_from_json
from    tensorflow.keras.optimizers           import  Adam, Nadam
from    tensorflow.keras.preprocessing.image  import  ImageDataGenerator
from    nbodykit.lab                          import  BigFileMesh
from    sklearn.model_selection               import  train_test_split
from    modelC                                import  prep_model_2D_C
from    preprocess                            import  pprocess
from    generator                             import  generator

In [11]:
np.random.seed(seed=314)

##  --  optimizers --                                                                                                                                                                                                      
adam  =   Adam(lr=1.e-4, amsgrad=True, clipnorm=1.)
nadam =  Nadam(lr=1.e-4, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004, clipnorm=1.)

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
def get_cosmos(nruns, nseeds, nslice):
  ##  Get list of available cosmologies.                                                                                                                                                                                   
  cosmos = np.loadtxt('cosmo.txt').tolist()
  labels = []

  ##  Generate labels where each cosmology is matched to nslice slices of nseeds sims.                                                                                                                                     
  for x in cosmos:
    for i in np.arange(nseeds):
      for j in np.arange(nslice):
        labels.append(x)

  labels = np.array(labels)
  nruns  = np.minimum(nruns, len(labels) * nseeds)

  return  nruns, labels

In [13]:
def plot(y, yhat, save=False):
  pl.clf()

  ymin  = 0.95 * y.min()
  ymax  = 1.05 * y.max()

  ys    = np.linspace(ymin, ymax, 10)

  pl.plot(ys,  ys, 'k-',   alpha=0.6)
  pl.plot(y, yhat, 'o', markersize=2)

  pl.xlabel(r'$f$')
  pl.ylabel(r'$\hat f$')

  pl.show()
  ##  pl.savefig('mlf.pdf')
  ##  os.system('xpdf mlf.pdf')

In [14]:
train, regress = True, True

optimizer      =  nadam

nseeds         =      9     ##  Number of random (seed) sims available in each cosmology.                                                                                                                                
nslice         =     16     ##  Sliced 3D sim. into _nslice_ (x, z) slices.                                                                                                                                              
nhot           =     10     ##  Supervised:  label sims by bin index in f;  nhot == # of bins.                                                                                                                           
nmesh          =    128
nruns          =    900     ##  Limit the total number of mocks input;  Set to e.g. 1e99 for all.                                                                                                                        
epochs         =      1     ##  Number of actual (keras) epochs.                                                                                                                                                        
valid_frac     =   0.15     ##  Split data X into fractions for train, validate/test.                                                                                                                                    
  
model          =  prep_model_2D_C(None, optimizer=optimizer, regress=regress)
nruns, labels  =  get_cosmos(nruns, nseeds, nslice)



Preparing model.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                524320    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 525,409
Trainable params: 525,409
Non-trainable params: 0
_________________________________________________________________


In [ ]:
if train:
    ##  Note:  horizontal and vertical flipping of 2D slices.                                                                                                                                                              
    train_gen = ImageDataGenerator(featurewise_center=False,\
                                   rotation_range=0,\
                                   width_shift_range=0.,\
                                   height_shift_range=0.,\
                                   horizontal_flip=True,\
                                   vertical_flip=True,\
                                   rescale=1.,\
                                   preprocessing_function=pprocess,\
                                   validation_split=valid_frac)  ##  Last Validation split.                                                                                                                                

    ##  Fit whitening params.                                                                                                                                                                                              
    ##  train_gen.fit(X)                                                                                                                                                                                                   

    ##  Image generator for continous creation with pre-processing;  steps_per_epoch=10 * len(X_train) / 32.                                                                                                               
    ##  train_gen.flow(X, y, batch_size=256, shuffle=True)                                                                                                                                                                 
    history = model.fit_generator(generator('./filelist.txt', batch_size=32, nmesh=nmesh, nslice=nslice, regress=regress, nhot=nhot),\
                                  steps_per_epoch=80, epochs=epochs, use_multiprocessing=True)

    ##  history = model.fit(X, y, validation_split=0.3, epochs=epochs)                                                                                                                                                     
    (X_test, y_test) = generator('./filelist.txt', batch_size=32, nmesh=nmesh, nslice=nslice, regress=regress, nhot=nhot)

    ##  Get current predictions.                                                                                                                                                                                           
    y_hat = model.predict(X_test)

    ##  Plot current prediction against truth (regression or supervised).                                                                                                                                                  
    plot(y_test, y_hat)

Instructions for updating:
Use tf.cast instead.


/global/home/users/mjwilson/.conda/envs/ntf/lib/python3.7/site-packages/numpy/core/numeric.py:538: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


14/80 [====>.........................] - ETA: 6:34 - loss: 0.0284 - acc: 0.0000e+00